# Amazon Logistics - El Buen Fin 2025 (Interfaz Interactiva)
## Universidad del SABES – Ingeniería Logística y Cadena de Valor
### Campus San Felipe · Octubre 2025

**PROYECTO**: Amazon Logistics – El Buen Fin 2025  
**Versión**: v3.7_Interactivo_Autonomo  
**Descripción**: Simulador interactivo de optimización logística

## 🔧 CONFIGURACIÓN INICIAL - EJECUTAR PRIMERO

In [ ]:
# Instalación de librerías
!pip install pulp networkx matplotlib ipywidgets --quiet
import math
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from pulp import *
import ipywidgets as widgets
from google.colab import output
output.enable_custom_widget_manager()

print("✅ Librerías instaladas y cargadas.")

## 👥 DATOS DE ALUMNOS INTEGRADOS

In [ ]:
# Crear lista de alumnos directamente en el código
alumnos_data = [
    # Administración y Desarrollo de Negocios
    {"matricula": "U2303057N0061", "nombre": "VICTOR GUADALUPE AMAYA GARCIA", "XX": 61},
    {"matricula": "U2303057N0138", "nombre": "NAYELI ARMADILLO PEREZ", "XX": 38},
    {"matricula": "U2303057N0065", "nombre": "ELIZABETH CAMARILLO ARMENTA", "XX": 65},
    {"matricula": "U2303057N0015", "nombre": "MARIA ESTRELLA CARRION PRADO", "XX": 15},
    {"matricula": "U2303057N0104", "nombre": "DAIRA CITLALLI CHUA AGUILAR", "XX": 4},
    {"matricula": "U2303057N0120", "nombre": "LUIS FERNANDO GODINEZ MARTINEZ", "XX": 20},
    {"matricula": "U2303057N0129", "nombre": "MARIA CONCEPCION YOCELIN GUERRA RAMOS", "XX": 29},
    {"matricula": "U2303057N0062", "nombre": "LUIS GERARDO GUTIERREZ ESCALERA", "XX": 62},
    {"matricula": "U2303057N0056", "nombre": "ARACELI DEL CARMEN LOPEZ ARZOLA", "XX": 56},
    {"matricula": "U2303057N0118", "nombre": "JUANA ISABEL MELENDEZ SOLIS", "XX": 18},
    {"matricula": "U2303057N0067", "nombre": "LIZETH GUADALUPE PADRON RODRIGUEZ", "XX": 67},
    {"matricula": "U2303057N0013", "nombre": "THELMA JULIETA ROCHA MARTINEZ", "XX": 13},
    {"matricula": "U2303057N0088", "nombre": "DANIELA GUADALUPE SALAZAR MARTINEZ", "XX": 88},
    {"matricula": "U2303057N0119", "nombre": "PERLA VALERIA TORRES SERVIN", "XX": 19},
    {"matricula": "U2303057N0005", "nombre": "KEYLA ZABALA RODARTE", "XX": 5},
    
    # Ingeniería Industrial
    {"matricula": "U2303057N0155", "nombre": "DENIEL DE HARO SEGURA", "XX": 55},
    {"matricula": "U2303057N0020", "nombre": "CENOBIO GARCIA ROMERO", "XX": 20},
    {"matricula": "U2303057N0080", "nombre": "FATIMA HERNANDEZ VILLEGAS", "XX": 80},
    {"matricula": "U2303057N0070", "nombre": "EYMI JOSELIN LUCIO RODRIGUEZ", "XX": 70},
    {"matricula": "U2303057N0012", "nombre": "JUAN SOLEDAD MACHUCA HERNANDEZ", "XX": 12},
    {"matricula": "U2303057N0021", "nombre": "MAYRA JAZMIN MEDELLIN MARQUEZ", "XX": 21},
    {"matricula": "U2303057N0046", "nombre": "LUIS ERNESTO MEJIA BANDA", "XX": 46},
    {"matricula": "U2303057N0086", "nombre": "PERLA DEL CARMEN MEJIA CANO", "XX": 86},
    {"matricula": "U2303057N0064", "nombre": "SANDY MENDOZA FLORES", "XX": 64},
    {"matricula": "U2203057N0070", "nombre": "JOSE MANUEL ORTIZ CHAVEZ", "XX": 70},
    {"matricula": "U2303057N0001", "nombre": "LUIS ALEJANDRO ORTIZ ORTIZ", "XX": 1},
    {"matricula": "U2303057N0110", "nombre": "JORGE ARMANDO PEREZ RANGEL", "XX": 10},
    {"matricula": "U2303057N0039", "nombre": "JHONATAN ISAAC ROCHA SALGADO", "XX": 39},
    {"matricula": "U2303057N0066", "nombre": "DAISA GUADALUPE RODRIGUEZ GARCIA", "XX": 66},
    {"matricula": "U2303057N0092", "nombre": "MANUEL EDUARDO RODRIGUEZ GARCIA", "XX": 92},
    {"matricula": "U2303057N0014", "nombre": "JORGE ALBERTO RODRIGUEZ PADILLA", "XX": 14},
    {"matricula": "U2303057N0041", "nombre": "EDGAR ROSALES MARQUEZ", "XX": 41},
    {"matricula": "U2303057N0077", "nombre": "ROSA ISELA VEGA CORREA", "XX": 77},
    {"matricula": "U2303057N0152", "nombre": "GEMMA YANEZ BERNAL", "XX": 52},
    
    # Mercadotecnia
    {"matricula": "U2303057N0027", "nombre": "TERESA DE JESUS CERVANTES NEAVE", "XX": 27},
    {"matricula": "U2303057N0050", "nombre": "CRISTIAN ANTONIO GARCIA LUNA", "XX": 50},
    {"matricula": "U2303057N0137", "nombre": "BLANCA GOMEZ ORTIZ", "XX": 37},
    {"matricula": "U2303057N0043", "nombre": "JOSE ALBERTO JIMENEZ", "XX": 43},
    {"matricula": "U2303057N0114", "nombre": "LUIS ANGEL RAMIREZ GARCIA", "XX": 14}
]

# Convertir a DataFrame
alumnos = pd.DataFrame(alumnos_data)
alumnos["display"] = alumnos["matricula"] + " - " + alumnos["nombre"]

print("✅ Lista de alumnos cargada:")
print(f"📊 Total de alumnos: {len(alumnos)}")
print("\n👥 Primeros 5 alumnos:")
for i, row in alumnos.head().iterrows():
    print(f"   {i+1}. {row['display']} (XX: {row['XX']})")

## 📊 DATOS BASE DEL SISTEMA LOGÍSTICO

In [ ]:
# Parámetros del sistema logístico
OFERTA_BASE = {
    'CDMX_Norte': 35000,
    'Monterrey':  22000,
    'Guadalajara':20000,
    'Queretaro':  18000,
    'Puebla':     15000,
    'Tijuana':    12000,
}

DEMANDA_BASE = {
    'Merida':   12000,
    'Cancun':   15000,
    'Toluca':   18000,
    'Leon':     14000,
    'SLP':      11000,
    'Veracruz': 13000,
}

COSTO_BASE_MILES = {
    'CDMX_Norte': {'Merida':142, 'Cancun':168, 'Toluca':8, 'Leon':45, 'SLP':52, 'Veracruz':48},
    'Monterrey': {'Merida':228, 'Cancun':245, 'Toluca':88, 'Leon':78, 'SLP':38, 'Veracruz':115},
    'Guadalajara': {'Merida':198, 'Cancun':215, 'Toluca':62, 'Leon':28, 'SLP':58, 'Veracruz':88},
    'Queretaro': {'Merida':158, 'Cancun':178, 'Toluca':18, 'Leon':22, 'SLP':35, 'Veracruz':52},
    'Puebla': {'Merida':128, 'Cancun':145, 'Toluca':22, 'Leon':52, 'SLP':62, 'Veracruz':28},
    'Tijuana': {'Merida':485, 'Cancun':498, 'Toluca':305, 'Leon':262, 'SLP':268, 'Veracruz':358},
}

COSTO_FIJO_TRAILER = 12500
PRESUPUESTO_MAX_DEFAULT = 2_850_000

print("✅ Datos base logísticos cargados:")
print(f"🏭 HUBs de distribución: {list(OFERTA_BASE.keys())}")
print(f"🏙️ Ciudades destino: {list(DEMANDA_BASE.keys())}")
print(f"💰 Presupuesto base: ${PRESUPUESTO_MAX_DEFAULT:,} MXN")
print(f"🚚 Costo fijo por trailer: ${COSTO_FIJO_TRAILER:,} MXN")

## 🎲 GENERADOR DE ESCENARIO PERSONALIZADO

In [ ]:
def generar_escenario_alumno(XX, demanda_extra_pct, capacidad_reduccion_pct, costo_ajuste_pct, presupuesto_max, escenario_nombre):
    """Genera escenario personalizado para el alumno"""
    
    mod10 = XX % 10
    mod7 = XX % 7
    mod15 = XX % 15
    mod5 = XX % 5

    # Factores base por XX
    factor_demanda_xx = 1 + (mod10 / 10)
    factor_oferta_xx = 1 + (mod7 / 10)
    factor_costos_xx = 0.85 + (mod15 / 100)

    print(f"🔢 Matrícula XX: {XX}")
    print(f"📈 Factores base - Demanda: +{(factor_demanda_xx-1)*100:.0f}%, Oferta: +{(factor_oferta_xx-1)*100:.0f}%, Costos: {(factor_costos_xx-1)*100:+.1f}%")

    # 1) Demanda personalizada
    demanda_mod = {}
    for dest, base in DEMANDA_BASE.items():
        demanda_mod[dest] = math.ceil(base * factor_demanda_xx * (1 + demanda_extra_pct))

    # 2) Oferta personalizada
    oferta_mod = {}
    for hub, base in OFERTA_BASE.items():
        oferta_mod[hub] = math.floor(base * factor_oferta_xx * (1 - capacidad_reduccion_pct))
        oferta_mod[hub] = max(100, oferta_mod[hub])

    # 3) Costos personalizados
    costos_mod = {}
    for hub, destinos in COSTO_BASE_MILES.items():
        costos_mod[hub] = {}
        for dest, costo_miles in destinos.items():
            costo_mxn = costo_miles * 1000.0
            costo_final = costo_mxn * factor_costos_xx * (1 + costo_ajuste_pct)
            costos_mod[hub][dest] = costo_final

    # 4) Ajustes por escenario
    if escenario_nombre.lower() == "optimista":
        for h in costos_mod:
            for d in costos_mod[h]:
                costos_mod[h][d] *= 0.90
        print("   🌟 Escenario Optimista: Costos reducidos 10%")

    elif escenario_nombre.lower() == "pesimista":
        for h in costos_mod:
            if "Cancun" in costos_mod[h]:
                costos_mod[h]["Cancun"] *= 1.25
            if "Veracruz" in costos_mod[h]:
                costos_mod[h]["Veracruz"] *= 1.25
        if "Monterrey" in oferta_mod:
            oferta_mod["Monterrey"] = math.floor(oferta_mod["Monterrey"] * 0.8)
        if "Puebla" in oferta_mod:
            oferta_mod["Puebla"] = math.floor(oferta_mod["Puebla"] * 0.8)
        print("   🌧️  Escenario Pesimista: Costos aumentados sureste, capacidad reducida noreste")

    elif escenario_nombre.lower() == "estrategico":
        BAJIO_CAP_ADICIONAL = 18000
        oferta_mod["BajioHub"] = BAJIO_CAP_ADICIONAL
        costos_mod["BajioHub"] = {
            "Leon": 22000 * (1 + costo_ajuste_pct),
            "SLP": 35000 * (1 + costo_ajuste_pct),
            "Toluca": 25000 * (1 + costo_ajuste_pct),
        }
        print("   🎯 Escenario Estratégico: Nuevo HUB Bajío agregado")

    # Resumen del escenario
    total_oferta = sum(oferta_mod.values())
    total_demanda = sum(demanda_mod.values())
    balance = total_oferta - total_demanda
    
    print(f"📊 Resumen escenario:")
    print(f"   • Oferta total: {total_oferta:,} unidades")
    print(f"   • Demanda total: {total_demanda:,} unidades")
    print(f"   • Balance: {'✅ SOSTENIBLE' if balance >= 0 else '⚠️  DEFICIT'} ({balance:+,} unidades)")
    print(f"   • Restricción especial: Tipo {mod5}")

    return oferta_mod, demanda_mod, costos_mod, mod5, presupuesto_max

print("✅ Generador de escenarios listo")

## ⚙️ SOLVER DE OPTIMIZACIÓN

In [ ]:
def optimizar(oferta, demanda, costos, mod5, presupuesto_max, costo_fijo_trailer=COSTO_FIJO_TRAILER):
    hubs = list(oferta.keys())
    destinos = list(demanda.keys())

    prob = LpProblem("Amazon_BuenFin2025_Interactivo", LpMinimize)

    # Crear rutas válidas
    rutas = []
    for h in hubs:
        for d in destinos:
            if h in costos and d in costos[h]:
                rutas.append((h, d))

    # Variables
    x = LpVariable.dicts("x", rutas, lowBound=0, cat="Integer")
    u = LpVariable.dicts("u", rutas, lowBound=0, upBound=1, cat="Binary")

    # Función objetivo
    total_cost = lpSum([
        costos[h][d] * x[(h, d)] + costo_fijo_trailer * u[(h, d)]
        for (h, d) in rutas
    ])
    prob += total_cost

    # Restricciones
    for h in hubs:
        prob += lpSum([x.get((h, d), 0) for d in destinos if (h, d) in x]) <= oferta[h], f"Capacidad_{h}"

    for d in destinos:
        prob += lpSum([x.get((h, d), 0) for h in hubs if (h, d) in x]) >= demanda[d], f"Demanda_{d}"

    for (h, d) in rutas:
        M = max(oferta[h], demanda[d])
        prob += x[(h, d)] <= M * u[(h, d)], f"ActivaRuta_{h}_{d}"

    prob += total_cost <= presupuesto_max, "PresupuestoMax"

    # Restricciones especiales basadas en mod5
    if mod5 == 0 and "Monterrey" in oferta:
        prob += lpSum([x.get(("Monterrey", d), 0) for d in destinos if ("Monterrey", d) in x]) <= oferta["Monterrey"] * 0.7
        print("   🎯 Restricción: Monterrey limitado al 70% de capacidad")

    if mod5 == 1 and "Queretaro" in oferta:
        prob += lpSum([x.get(("Queretaro", d), 0) for d in destinos if ("Queretaro", d) in x]) >= 1
        print("   🎯 Restricción: Querétaro debe operar")

    if mod5 == 3 and "CDMX_Norte" in oferta and "Cancun" in demanda:
        if ("CDMX_Norte", "Cancun") in x:
            prob += x[("CDMX_Norte", "Cancun")] >= 0.30 * demanda["Cancun"]
            print("   🎯 Restricción: CDMX debe surtir 30% de Cancún")

    if mod5 == 4 and "Tijuana" in oferta:
        rutas_tijuana = [("Tijuana", d) for d in destinos if ("Tijuana", d) in u]
        if rutas_tijuana:
            prob += lpSum([u[r] for r in rutas_tijuana]) <= 3
            print("   🎯 Restricción: Tijuana máximo 3 rutas")

    # Resolver
    print("   🔍 Resolviendo modelo de optimización...")
    prob.solve(PULP_CBC_CMD(msg=0))

    if LpStatus[prob.status] != "Optimal":
        raise ValueError(f"❌ No se encontró solución óptima. Status: {LpStatus[prob.status]}")

    # Extraer resultados
    asignaciones = {}
    for (h, d) in rutas:
        val = x[(h, d)].varValue
        if val and val > 0.5:
            asignaciones[(h, d)] = int(val)

    uso_hub = {h: sum(asignaciones.get((h, d), 0) for d in destinos) for h in hubs}
    utilizacion_hub = {h: (uso_hub[h] / oferta[h] if oferta[h] > 0 else 0) for h in hubs}

    return {
        "status": LpStatus[prob.status],
        "costo_total": value(total_cost),
        "asignaciones": asignaciones,
        "utilizacion": utilizacion_hub,
        "oferta": oferta,
        "demanda": demanda,
        "presupuesto": presupuesto_max,
        "total_unidades": sum(asignaciones.values()),
        "rutas_activas": len(asignaciones)
    }

print("✅ Solver de optimización listo")

## 📊 VISUALIZACIÓN DE LA RED LOGÍSTICA

In [ ]:
def graficar_red(resultado):
    asign = resultado["asignaciones"]
    oferta = resultado["oferta"]
    demanda = resultado["demanda"]

    G = nx.DiGraph()
    pos = {}

    # Nodos HUBs
    hubs = list(oferta.keys())
    for i, h in enumerate(hubs):
        pos[h] = (0, 1 - (i+1)/(len(hubs)+1))
        G.add_node(h, tipo="hub")

    # Nodos destinos
    destinos = list(demanda.keys())
    for j, d in enumerate(destinos):
        pos[d] = (1, 1 - (j+1)/(len(destinos)+1))
        G.add_node(d, tipo="dest")

    # Arcos
    max_flujo = max(asign.values()) if asign else 1
    for (h, d), f in asign.items():
        G.add_edge(h, d, weight=f)
        G[h][d]['ancho'] = (2 + 6*f/max_flujo)

    # Dibujar
    plt.figure(figsize=(16, 10))
    
    hubs_nodes = [n for n in G.nodes() if G.nodes[n]["tipo"]=="hub"]
    dest_nodes = [n for n in G.nodes() if G.nodes[n]["tipo"]=="dest"]

    # Dibujar nodos
    nx.draw_networkx_nodes(G, pos, nodelist=hubs_nodes, node_color="#1565c0", 
                          node_size=3000, node_shape="s", edgecolors="black", linewidths=2)
    nx.draw_networkx_nodes(G, pos, nodelist=dest_nodes, node_color="#ff9900", 
                          node_size=2800, node_shape="o", edgecolors="black", linewidths=2)

    # Bordes con ancho proporcional al flujo
    edges = list(G.edges())
    widths = [G[u][v]['ancho'] for u,v in edges] if edges else [2]
    nx.draw_networkx_edges(G, pos, edge_color="#424242", width=widths, 
                          arrows=True, arrowsize=25, alpha=0.7)

    # Etiquetas de nodos
    labels = {}
    util = resultado["utilizacion"]
    for h in hubs_nodes:
        uso_pct = util.get(h,0)*100
        labels[h] = f"{h}\n{uso_pct:.0f}% uso"
    for d in dest_nodes:
        labels[d] = f"{d}\n{demanda[d]:,}"

    nx.draw_networkx_labels(G, pos, labels, font_weight="bold", font_size=10)
    
    # Etiquetas de arcos
    edge_labels = {(h,d): f"{f:,}" for (h,d), f in asign.items()}
    nx.draw_networkx_edge_labels(G, pos, edge_labels, font_size=9)

    plt.title(f"Amazon México - El Buen Fin 2025\n"
              f"Costo Total: ${resultado['costo_total']:,.0f} MXN | "
              f"Unidades: {resultado['total_unidades']:,} | "
              f"Rutas: {resultado['rutas_activas']}", 
              fontsize=16, fontweight="bold", pad=20)
    plt.axis("off")
    plt.tight_layout()
    plt.show()

print("✅ Módulo de visualización listo")

## 🎮 INTERFAZ INTERACTIVA

In [ ]:
print("🎯 Configurando interfaz interactiva...")

# Widgets
dropdown_alumno = widgets.Dropdown(
    options=list(alumnos["display"]),
    description="👤 Alumno:",
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px')
)

slider_demanda = widgets.FloatSlider(
    value=0.20, min=0.0, max=0.50, step=0.05,
    description="📈 Demanda extra:",
    style={'description_width': 'initial'},
    readout_format='.0%'
)

slider_capacidad = widgets.FloatSlider(
    value=0.10, min=0.0, max=0.50, step=0.05,
    description="🏭 Reducción capacidad:",
    style={'description_width': 'initial'},
    readout_format='.0%'
)

slider_costos = widgets.FloatSlider(
    value=0.15, min=-0.30, max=0.50, step=0.05,
    description="💰 Ajuste costos:",
    style={'description_width': 'initial'},
    readout_format='.0%'
)

slider_presupuesto = widgets.IntSlider(
    value=PRESUPUESTO_MAX_DEFAULT,
    min=1_500_000,
    max=4_000_000,
    step=50_000,
    description="💵 Presupuesto (MXN):",
    style={'description_width': 'initial'},
    readout_format='d'
)

dropdown_escenario = widgets.Dropdown(
    options=["Optimista", "Pesimista", "Estrategico"],
    value="Optimista",
    description="🌍 Escenario:",
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='300px')
)

boton_run = widgets.Button(
    description="🚀 Resolver Escenario",
    button_style='success',
    layout=widgets.Layout(width='200px', height='40px')
)

output_area = widgets.Output()

def ejecutar_callback(_):
    with output_area:
        output_area.clear_output()
        
        try:
            print("🔄 Procesando escenario...")
            
            # Obtener datos del alumno
            row = alumnos[alumnos["display"] == dropdown_alumno.value].iloc[0]
            alumno_display = row["display"]
            XX = int(row["XX"])

            print(f"👤 Alumno: {alumno_display}")
            print(f"🔢 Matrícula XX: {XX}")
            
            # Generar escenario
            oferta_mod, demanda_mod, costos_mod, mod5, presupuesto_aplicado = generar_escenario_alumno(
                XX,
                slider_demanda.value,
                slider_capacidad.value,
                slider_costos.value,
                slider_presupuesto.value,
                dropdown_escenario.value
            )

            # Optimizar
            resultado = optimizar(oferta_mod, demanda_mod, costos_mod, mod5, presupuesto_aplicado)

            # Mostrar resultados detallados
            print("\n" + "="*70)
            print("📊 REPORTE EJECUTIVO - AMAZON MÉXICO EL BUEN FIN 2025")
            print("="*70)
            print(f"👤 ALUMNO: {alumno_display}")
            print(f"🎯 ESCENARIO: {dropdown_escenario.value}")
            print(f"💰 COSTO TOTAL: ${resultado['costo_total']:,.0f} MXN")
            print(f"📦 UNIDADES TRANSPORTADAS: {resultado['total_unidades']:,}")
            print(f"🛣️  RUTAS ACTIVAS: {resultado['rutas_activas']}")
            print(f"💵 PRESUPUESTO: ${resultado['presupuesto']:,.0f} MXN")
            
            print(f"\n🏭 UTILIZACIÓN DE HUBS:")
            for hub, util in sorted(resultado['utilizacion'].items(), key=lambda x: x[1], reverse=True):
                estado = "⚡ ALTA" if util > 0.8 else "✅ ÓPTIMA" if util > 0.5 else "⚠️  BAJA"
                print(f"   • {hub:<12}: {util:>6.1%} {estado}")
            
            print(f"\n🚚 PRINCIPALES FLUJOS:")
            flujos_ordenados = sorted(resultado['asignaciones'].items(), key=lambda x: x[1], reverse=True)[:8]
            for (origen, destino), unidades in flujos_ordenados:
                porcentaje = (unidades / resultado['total_unidades']) * 100
                print(f"   • {origen:<12} → {destino:<10}: {unidades:>6,} unidades ({porcentaje:.1f}%)")

            # Métricas de eficiencia
            if resultado['total_unidades'] > 0:
                costo_por_unidad = resultado['costo_total'] / resultado['total_unidades']
                print(f"\n📈 MÉTRICAS DE EFICIENCIA:")
                print(f"   • Costo por unidad: ${costo_por_unidad:,.2f} MXN")
                print(f"   • Eficiencia presupuestaria: {(resultado['costo_total']/resultado['presupuesto'])*100:.1f}%")

            # Gráfico
            print(f"\n📊 Generando visualización de la red...")
            graficar_red(resultado)
            
            print("\n✅ SIMULACIÓN COMPLETADA EXITOSAMENTE")
            
        except Exception as e:
            print(f"❌ Error en la simulación: {str(e)}")
            print("💡 Sugerencias:")
            print("   - Intenta aumentar el presupuesto")
            print("   - Reduce la demanda extra")
            print("   - Aumenta la capacidad disponible")
            print("   - Prueba con otro alumno o escenario")

boton_run.on_click(ejecutar_callback)

# Mostrar interfaz
ui = widgets.VBox([
    widgets.HTML(
        "<h1>🚚 Amazon Logistics - El Buen Fin 2025</h1>"
        "<h3>Universidad del SABES - Ingeniería Logística y Cadena de Valor</h3>"
        "<p><b>Simulador Interactivo de Optimización Logística</b></p>"
        "<p>Selecciona tu nombre, ajusta los parámetros y resuelve tu escenario personalizado.</p>"
        "<hr>"
    ),
    dropdown_alumno,
    widgets.HBox([dropdown_escenario, slider_presupuesto]),
    slider_demanda,
    slider_capacidad,
    slider_costos,
    widgets.HBox([boton_run]),
    output_area
])

display(ui)
print("✅ Interfaz lista. Selecciona tu nombre y haz clic en 'Resolver Escenario'")